In [28]:
import sys
import os
import time
import pandas as pd
import contextlib
import numpy as np
# Importing the main code.
module_path = os.path.abspath(os.path.join('../src/ocloc'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ocloc import ProcessingParameters, ClockDrift
import jackknife_tools as jk
import pickle

# Parameters used for the refined inversion were:

params2 = ProcessingParameters(
                 freqmin = 0.2, # Low freq. for the bandpass filter
                 freqmax = 0.4, # High freq. for the bandpass filter 
                 ref_vel = 4500, # m/s
                 dist_trh = 2.5, # Minimum station separation in terms of wavelength
                 snr_trh = 30, # Signal-to-noise ratio threshold
                 noise_st = 240, # start of the noise window.
                 dt_err = 0.004, # Sampling interval needs to be multiple of this value.
                 resp_details = False)

In [24]:
path_2_cd = ("precomputed_clockdrift_objects/ClockDrift_refined_inversion_" + 
             "snr_trh_30__dist_trh_2.5.obj")
with open(path_2_cd, 'rb') as f:
    cd = pickle.load(f) 


In [54]:
# Function to substrac values from a dictionary using another dictionary with same keys.
# the values are a list of values.
def subtract_dict(dict1, dict2):
    """Subtract values from a dictionary using another dictionary with same keys.
    Parameters
    ----------
    dict1 : dict
        Dictionary with values to be subtracted.
    dict2 : dict
        Dictionary with values to be subtracted.
    Returns
    -------
        dict
        Dictionary with values subtracted.
    """
    for k in dict1:
        dict1[k][-1] = dict1[k][-1] - dict2[k][-1]

    return dict1

def calculate_pseudo_values(cd, pseudo_cd):
    """Calculate pseudo values for a clock drift object.
    Parameters
    ----------
    cd : ClockDrift
        Clock drift object.
    pseudo_cd : ClockDrift
        Clock drift object with pseudo values.
    
    Returns
    -------
    pseudo_cd_a_vals : dict
        Dictionary with pseudo values.
    pseudo_cd_b_vals :
        Dictionary with pseudo values.
    """

    a_vals = jk.get_last_a_value(cd)
    b_vals = jk.get_last_b_value(cd)

    pseudo_cd_a_vals = jk.get_last_a_value(pseudo_cd)
    pseudo_cd_b_vals = jk.get_last_b_value(pseudo_cd)

    for station in a_vals:
        pseudo_cd_a_vals[station] = [(a_vals[station][-1] 
                                     - pseudo_cd_a_vals[station][-1])]
        pseudo_cd_b_vals[station] = [(b_vals[station][-1]
                                     - pseudo_cd_b_vals[station][-1])]

    return pseudo_cd_a_vals, pseudo_cd_b_vals


In [80]:
snr_trh = 30
dist_trh = 2.5

cd = jk.load_clockdrift(snr_trh=snr_trh, dist_trh=dist_trh, 
                     dir_ClockDrifts="precomputed_clockdrift_objects/")

# Remove the correlation in cd.correlations that have 
# a nan value in t_app[-1].
cd = jk.remove_nan_corr(cd)

# build the matrix A and the vector Tobs.
cd.build_matrices()

# Create an empty list to store the results
results = []

# Iterate over the rows of A and T
for index in range(20): #len(cd.correlations)):
    
    pseudo_cd = cd.copy()
    pseudo_cd.correlations[index].t_app[-1] = np.nan
    pseudo_cd = jk.remove_nan_corr(pseudo_cd)

    # Do not show output from calling a function.
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        pseudo_cd.build_matrices()
        pseudo_cd.solve_eq()

    # calculate the pseudo-value for the left-out observation
    pseudo_values_a, pseudo_values_b = calculate_pseudo_values(cd, pseudo_cd)

    if index == 0:
        a_vals = pseudo_values_a
        b_vals = pseudo_values_b
    else:
        a_vals = jk.combine_dicts(a_vals, pseudo_values_a)
        b_vals = jk.combine_dicts(b_vals, pseudo_values_b)
    station = "O10"

    print(pseudo_values_a[station][-1])
# Calculate the mean of the pseudo-values.
for station in a_vals:
    mean_a = np.mean(a_vals[station])
    mean_b = np.mean(b_vals[station])


-0.000360507676411
-0.000360507676411
-0.000359990316683
-0.000359990316683
-0.000359244128976
-0.000359244128976
-0.000358664854413
-0.000358664854413
-0.000360925991062
-0.000360925991062
-0.000360172219282
-0.000360172219282
-0.000362139404886
-0.000362139404886
-0.000360406406834
-0.000360406406834
-0.000360191247489
-0.000360191247489
-0.000361713716389
-0.000361713716389
-0.000367939012191
-0.000367939012191
-0.000404849222899
-0.000404849222899
-0.00035976272405
-0.00035976272405
-0.00035978949536
-0.00035978949536
-0.000359340063195
-0.000359340063195
-0.000360080828487
-0.000360080828487
-0.000359483115072
-0.000359483115072
-0.0003599863508
-0.0003599863508
-0.000360197819032
-0.000360197819032
-0.000359869674052
-0.000359869674052


In [68]:
-0.00097399806446048719+0.0010879047937566741

0.00011390672929618691

In [58]:
# Plot the a values.
station="O01"
jk.plot_hist(a_vals[station], "a", xlabel="a", ylabel="Frequency")


100